In [38]:
import numpy as np
import cv2  # OpenCVライブラリ

import matplotlib.pyplot as plt 
%matplotlib inline

import torch

import pickle

import torch.utils.data as data
from itertools import product as product

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from torch.autograd import Function

In [2]:
def group_annotation_by_class(dataset):
    true_case_stat = {}
    all_gt_boxes = {}
    all_difficult_cases = {}
    for i in range(len(dataset)):
        image_id, annotation = dataset.get_annotation(i)
        gt_boxes, classes, is_difficult = annotation
        gt_boxes = torch.from_numpy(gt_boxes)
        for i, difficult in enumerate(is_difficult):
            class_index = int(classes[i])
            gt_box = gt_boxes[i]
            if not difficult:
                true_case_stat[class_index] = true_case_stat.get(class_index, 0) + 1

            if class_index not in all_gt_boxes:
                all_gt_boxes[class_index] = {}
            if image_id not in all_gt_boxes[class_index]:
                all_gt_boxes[class_index][image_id] = []
            all_gt_boxes[class_index][image_id].append(gt_box)
            if class_index not in all_difficult_cases:
                all_difficult_cases[class_index]={}
            if image_id not in all_difficult_cases[class_index]:
                all_difficult_cases[class_index][image_id] = []
            all_difficult_cases[class_index][image_id].append(difficult)

    for class_index in all_gt_boxes:
        for image_id in all_gt_boxes[class_index]:
            all_gt_boxes[class_index][image_id] = torch.stack(all_gt_boxes[class_index][image_id])
    for class_index in all_difficult_cases:
        for image_id in all_difficult_cases[class_index]:
            all_gt_boxes[class_index][image_id] = torch.tensor(all_gt_boxes[class_index][image_id])
    return true_case_stat, all_gt_boxes, all_difficult_cases


def compute_average_precision_per_class(num_true_cases, gt_boxes, difficult_cases,
                                        prediction_file, iou_threshold, use_2007_metric):
    with open(prediction_file) as f:
        image_ids = []
        boxes = []
        scores = []
        for line in f:
            t = line.rstrip().split(" ")
            image_ids.append(t[0])
            scores.append(float(t[1]))
            box = torch.tensor([float(v) for v in t[2:]]).unsqueeze(0)
            box -= 1.0  # convert to python format where indexes start from 0
            boxes.append(box)
        scores = np.array(scores)
        sorted_indexes = np.argsort(-scores)
        boxes = [boxes[i] for i in sorted_indexes]
        image_ids = [image_ids[i] for i in sorted_indexes]
        true_positive = np.zeros(len(image_ids))
        false_positive = np.zeros(len(image_ids))
        matched = set()
        for i, image_id in enumerate(image_ids):
            box = boxes[i]
            if image_id not in gt_boxes:
                false_positive[i] = 1
                continue

            gt_box = gt_boxes[image_id]
            ious = box_utils.iou_of(box, gt_box)
            max_iou = torch.max(ious).item()
            max_arg = torch.argmax(ious).item()
            if max_iou > iou_threshold:
                if difficult_cases[image_id][max_arg] == 0:
                    if (image_id, max_arg) not in matched:
                        true_positive[i] = 1
                        matched.add((image_id, max_arg))
                    else:
                        false_positive[i] = 1
            else:
                false_positive[i] = 1

    true_positive = true_positive.cumsum()
    false_positive = false_positive.cumsum()
    precision = true_positive / (true_positive + false_positive)
    recall = true_positive / num_true_cases
    if use_2007_metric:
        return measurements.compute_voc2007_average_precision(precision, recall)
    else:
        return measurements.compute_average_precision(precision, recall)

In [3]:
# import dataset
from utils.dataset import VOCDataset, DatasetTransform, make_datapath_list, Anno_xml2list, od_collate_fn

In [32]:
# set your VOCdevkit path!
vocpath = "../VOCdevkit/VOC2007"
DEVKIT_PATH = "../VOCdevkit/"
SET = "test"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(vocpath)


In [5]:
val_img_list[0]

'../VOCdevkit/VOC2007/JPEGImages/000001.jpg'

In [11]:
image_index = []
for l in val_img_list:
    image_index.append(l[-10:-4])

image_index[0]

'000001'

In [12]:
class_names = ['aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor']
color_mean = (104, 117, 123)  # (BGR)の色の平均値
input_size = 300  # 画像のinputサイズを300×300にする

## DatasetTransformを適応
transform = DatasetTransform(input_size, color_mean)
transform_anno = Anno_xml2list(class_names)

In [13]:
val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DatasetTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(class_names))

In [14]:
val_dataloader = data.DataLoader(
    val_dataset, batch_size=1, shuffle=False, collate_fn=od_collate_fn, num_workers=8)

# set up model

In [15]:
from utils.ssd_model import SSD

voc_classes = ['aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor']

# SSD300の設定
ssd_cfg = {
    'num_classes': 21,  # 背景クラスを含めた合計クラス数
    'input_size': 300,  # 画像の入力サイズ
    'bbox_aspect_num': [4, 6, 6, 6, 4, 4],  # 出力するDBoxのアスペクト比の種類
    'feature_maps': [38, 19, 10, 5, 3, 1],  # 各sourceの画像サイズ
    'steps': [8, 16, 32, 64, 100, 300],  # DBOXの大きさを決める
    'min_sizes': [30, 60, 111, 162, 213, 264],  # DBOXの大きさを決める
    'max_sizes': [60, 111, 162, 213, 264, 315],  # DBOXの大きさを決める
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
}

# SSDネットワークモデル
net = SSD(phase="inference", cfg=ssd_cfg).eval()

# SSDの学習済みの重みを設定
net_weights = torch.load('./weights/ssd300_130.pth',
                         map_location={'cuda:0': 'cpu'})

net.load_state_dict(net_weights)

print('loaded the trained weights')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("using:", device)

net = net.to(device)

loaded the trained weights
using: cuda:0


In [16]:
all_imgs = []

classes = {}

bbox_threshold = 0.05

# define detections
all_boxes = [[[] for _ in range(len(val_img_list))]
               for _ in range(21)]
empty_array = np.transpose(np.array([[],[],[],[],[]]), (1,0))

In [17]:
from utils.ssd_predict_show import SSDPredictShow
ssd = SSDPredictShow(eval_categories=voc_classes, net=net, device=device)

cuda:0


# infer images

In [69]:
for i, imp in enumerate(val_img_list):
    rgb_img, predict_bbox, pre_dict_label_index, scores = ssd.ssd_predict(imp, data_confidence_level=0.05)
    
    for cls in range(21):
        box = []
        for j,pred in enumerate(predict_bbox):
            if cls == pre_dict_label_index[j]:
                box.append([scores[j], pred[0],pred[1],pred[2],pred[3]])
        if not box == []:
            all_boxes[cls][i] = box
        else:
            all_boxes[cls][i] = empty_array
    if i%1000==0:
        print("iter:", i)

iter: 0
iter: 1000
iter: 2000
iter: 3000
iter: 4000


In [85]:
all_boxes[7][0:10]

[array([], shape=(0, 5), dtype=float64),
 array([], shape=(0, 5), dtype=float64),
 array([], shape=(0, 5), dtype=float64),
 array([], shape=(0, 5), dtype=float64),
 array([], shape=(0, 5), dtype=float64),
 array([], shape=(0, 5), dtype=float64),
 array([], shape=(0, 5), dtype=float64),
 [[0.9321504,
   145.5070674419403,
   111.48892092704773,
   322.5487470626831,
   299.3372104167938],
  [0.3153717,
   147.28742837905884,
   54.05418115854263,
   239.3389642238617,
   120.71989560127258]],
 array([], shape=(0, 5), dtype=float64),
 array([], shape=(0, 5), dtype=float64)]

# eval accuracy

In [70]:
# eval function
def voc_eval(detpath,
             annopath,
             imagesetfile,
             classname,
             cachedir,
             ovthresh=0.5,
             use_07_metric=False):
  """
  rec, prec, ap = voc_eval(detpath,
                              annopath,
                              imagesetfile,
                              classname,
                              [ovthresh],
                              [use_07_metric])
  Top level function that does the PASCAL VOC evaluation.
  detpath: Path to detections
      detpath.format(classname) should produce the detection results file.
  annopath: Path to annotations
      annopath.format(imagename) should be the xml annotations file.
  imagesetfile: Text file containing the list of images, one image per line.
  classname: Category name (duh)
  cachedir: Directory for caching the annotations
  [ovthresh]: Overlap threshold (default = 0.5)
  [use_07_metric]: Whether to use VOC07's 11 point AP computation
      (default False)
  """
  # assumes detections are in detpath.format(classname)
  # assumes annotations are in annopath.format(imagename)
  # assumes imagesetfile is a text file with each line an image name
  # cachedir caches the annotations in a pickle file

  # first load gt
  if not os.path.isdir(cachedir):
    os.mkdir(cachedir)
  cachefile = os.path.join(cachedir, '%s_annots.pkl' % imagesetfile)
  # read list of images
  with open(imagesetfile, 'r') as f:
    lines = f.readlines()
  imagenames = [x.strip() for x in lines]

  if not os.path.isfile(cachefile):
    # load annotations
    recs = {}
    for i, imagename in enumerate(imagenames):
      recs[imagename] = parse_rec(annopath.format(imagename))
      if i % 100 == 0:
        print('Reading annotation for {:d}/{:d}'.format(
          i + 1, len(imagenames)))
    # save
    #print('Saving cached annotations to {:s}'.format(cachefile))
    #with open(cachefile, 'wb') as f:
    #  pickle.dump(recs, f)
  else:
    # load
    with open(cachefile, 'rb') as f:
      try:
        recs = pickle.load(f)
      except:
        recs = pickle.load(f, encoding='bytes')

  # extract gt objects for this class
  class_recs = {}
  npos = 0
  for imagename in imagenames:
    R = [obj for obj in recs[imagename] if obj['name'] == classname]
    bbox = np.array([x['bbox'] for x in R])
    difficult = np.array([x['difficult'] for x in R]).astype(np.bool)
    det = [False] * len(R)
    npos = npos + sum(~difficult)
    class_recs[imagename] = {'bbox': bbox,
                             'difficult': difficult,
                             'det': det}

  # read dets
  detfile = detpath.format(classname)
  with open(detfile, 'r') as f:
    lines = f.readlines()

  splitlines = [x.strip().split(' ') for x in lines]
  image_ids = [x[0] for x in splitlines]
  confidence = np.array([float(x[1]) for x in splitlines])
  BB = np.array([[float(z) for z in x[2:]] for x in splitlines])

  nd = len(image_ids)
  tp = np.zeros(nd)
  fp = np.zeros(nd)

  if BB.shape[0] > 0:
    # sort by confidence
    sorted_ind = np.argsort(-confidence)
#    sorted_scores = np.sort(-confidence)
    BB = BB[sorted_ind, :]
    image_ids = [image_ids[x] for x in sorted_ind]

    # go down dets and mark TPs and FPs
    for d in range(nd):
      id = image_ids[d][-10:-4]
      #print(id)
      # catch bad detections
      try:
          R = class_recs[id]
      except:
        #print("det not found")
        continue
        
      bb = BB[d, :].astype(float)
      ovmax = -np.inf
      BBGT = R['bbox'].astype(float)

      if BBGT.size > 0:
        # compute overlaps
        # intersection
        ixmin = np.maximum(BBGT[:, 0], bb[0])
        iymin = np.maximum(BBGT[:, 1], bb[1])
        ixmax = np.minimum(BBGT[:, 2], bb[2])
        iymax = np.minimum(BBGT[:, 3], bb[3])
        iw = np.maximum(ixmax - ixmin + 1., 0.)
        ih = np.maximum(iymax - iymin + 1., 0.)
        inters = iw * ih

        # union
        uni = ((bb[2] - bb[0] + 1.) * (bb[3] - bb[1] + 1.) +
               (BBGT[:, 2] - BBGT[:, 0] + 1.) *
               (BBGT[:, 3] - BBGT[:, 1] + 1.) - inters)

        overlaps = inters / uni
        ovmax = np.max(overlaps)
        jmax = np.argmax(overlaps)

      if ovmax > ovthresh:
        if not R['difficult'][jmax]:
          if not R['det'][jmax]:
            tp[d] = 1.
            R['det'][jmax] = 1
          else:
            fp[d] = 1.
      else:
        fp[d] = 1.

  # compute precision recall
  fp = np.cumsum(fp)
  tp = np.cumsum(tp)
  rec = tp / float(npos)
  # avoid divide by zero in case the first detection matches a difficult
  # ground truth
  prec = tp / np.maximum(tp + fp, np.finfo(np.float64).eps)
  ap = voc_ap(rec, prec, use_07_metric)

  return rec, prec, ap

In [79]:
pascal_classes = np.asarray(['aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor'])
PASCAL_CLASSES = pascal_classes

# write out detections for evaluation

In [90]:
import os 
def get_voc_results_file_template(cls):
        # VOCdevkit/results/VOC2007/Main/<comp_id>_det_test_aeroplane.txt
        filename = 'det_' + "val" + '_'+cls+'.txt'
        filedir = os.path.join(DEVKIT_PATH, 'results', 'VOC2007', 'Main')
        if not os.path.exists(filedir):
            os.makedirs(filedir)
        path = os.path.join(filedir, filename)
        return path


def write_voc_results_file(pascal_classes, all_boxes, image_index):
        for cls_ind, cls in enumerate(pascal_classes):
            if cls == '__background__':
                continue
            print('Writing {} VOC results file'.format(cls))
            filename = get_voc_results_file_template(cls)
            with open(filename, 'wt') as f:
                for im_ind, index in enumerate(image_index):
                    dets = np.asarray(all_boxes[cls_ind][im_ind])
                    if dets == []:
                        continue
                    # the VOCdevkit expects 1-based indices
                    for k in range(dets.shape[0]):
                        #print(dets[k, 0])
                        f.write('{:s} {:.3f} {:.1f} {:.1f} {:.1f} {:.1f}\n'.
                                format(index, dets[k, 0],
                                       dets[k, 1] + 1, dets[k, 2] + 1,
                                       dets[k, 3] + 1, dets[k, 4] + 1))
import xml.etree.ElementTree as ET
def parse_rec(filename):
  """ Parse a PASCAL VOC xml file """
  tree = ET.parse(filename)
  objects = []
  for obj in tree.findall('object'):
    obj_struct = {}
    obj_struct['name'] = obj.find('name').text
    obj_struct['pose'] = obj.find('pose').text
    obj_struct['truncated'] = int(obj.find('truncated').text)
    obj_struct['difficult'] = int(obj.find('difficult').text)
    bbox = obj.find('bndbox')
    obj_struct['bbox'] = [int(bbox.find('xmin').text),
                          int(bbox.find('ymin').text),
                          int(bbox.find('xmax').text),
                          int(bbox.find('ymax').text)]
    objects.append(obj_struct)

  return objects
def voc_ap(rec, prec, use_07_metric=False):
  """ ap = voc_ap(rec, prec, [use_07_metric])
  Compute VOC AP given precision and recall.
  If use_07_metric is true, uses the
  VOC 07 11 point method (default:False).
  """
  if use_07_metric:
    # 11 point metric
    ap = 0.
    for t in np.arange(0., 1.1, 0.1):
      if np.sum(rec >= t) == 0:
        p = 0
      else:
        p = np.max(prec[rec >= t])
      ap = ap + p / 11.
  else:
    # correct AP calculation
    # first append sentinel values at the end
    mrec = np.concatenate(([0.], rec, [1.]))
    mpre = np.concatenate(([0.], prec, [0.]))

    # compute the precision envelope
    for i in range(mpre.size - 1, 0, -1):
      mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

    # to calculate area under PR curve, look for points
    # where X axis (recall) changes value
    i = np.where(mrec[1:] != mrec[:-1])[0]

    # and sum (\Delta recall) * prec
    ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
  return ap

In [91]:
write_voc_results_file(pascal_classes, all_boxes, val_img_list) #image_index)

Writing aeroplane VOC results file
0.0933142676949501
0.9994332194328308
0.0816735103726387
0.22935722768306732
0.11644931137561798
0.9821674823760986
0.6123923063278198
0.10524306446313858
0.08037786185741425
0.05972050875425339
0.05662132799625397
0.9989193677902222
0.9487029314041138
0.4712750017642975
0.14790396392345428
0.13128109276294708
0.12669406831264496
0.09990116208791733
0.07547712326049805
0.06903485208749771
0.9784066081047058
0.1222832128405571
0.17122285068035126
0.9986417889595032
0.08214888721704483
0.8642976880073547
0.6956445574760437
0.17587749660015106
0.09057936817407608
0.9815861582756042
0.07202169299125671
0.07137652486562729
0.9656450748443604
0.3680207133293152
0.08229974657297134
0.06287485361099243
0.05772753059864044
0.9960888624191284
0.09402178227901459
0.9200940728187561
0.06705419719219208
0.9961901903152466
0.2156929224729538
0.13646972179412842
0.08849246799945831
0.08700250089168549
0.07982398569583893
0.06601639837026596
0.06341028213500977
0.061

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


0.46255815029144287
0.19929566979408264
0.19593244791030884
0.18180766701698303
0.13171283900737762
0.11048665642738342
0.10948921740055084
0.09466422349214554
0.0859571248292923
0.0806201845407486
0.07633959501981735
0.0747314989566803
0.07420375943183899
0.0724322497844696
0.06995796412229538
0.0697372630238533
0.06732599437236786
0.06507151573896408
0.055548302829265594
0.050173573195934296
0.050568919628858566
0.057216331362724304
0.3423402011394501
0.07135001569986343
0.9992480874061584
0.08035945147275925
0.05975071340799332
0.05608562007546425
0.2520197033882141
0.07362717390060425
0.9994685053825378
0.13783912360668182
0.0556567944586277
0.0779719203710556
0.1906304955482483
0.055775027722120285
0.06410907208919525
0.0537598691880703
0.06225355714559555
0.07613686472177505
0.978627622127533
0.050674572587013245
0.1251557618379593
0.058701321482658386
0.09382977336645126
0.29511523246765137
0.46568039059638977
0.09101459383964539
0.06847789883613586
0.4335371255874634
0.14325740

0.09146285057067871
0.08285492658615112
0.5012680292129517
0.9524544477462769
0.0886935368180275
0.0613938570022583
0.05808927118778229
0.05336574837565422
0.0521635040640831
0.10615930706262589
0.07311465591192245
0.05672586336731911
0.9966541528701782
0.5444649457931519
0.13483533263206482
0.11708061397075653
0.055216141045093536
0.17029324173927307
0.07228268682956696
0.0625913143157959
0.05712013691663742
0.05507085844874382
0.05096270143985748
0.4990648925304413
0.6827404499053955
0.6126318573951721
0.30512648820877075
0.1989665925502777
0.12503203749656677
0.1038273498415947
0.09074462950229645
0.0876895859837532
0.07955913990736008
0.06576251238584518
0.06089038401842117
0.060511548072099686
0.05331622064113617
0.0525595098733902
0.3578799068927765
0.16539280116558075
0.05730728805065155
0.2201855182647705
0.8695750832557678
0.11519922316074371
0.99824059009552
0.7335830330848694
0.13719545304775238
0.052473969757556915
0.4358300268650055
0.2942984998226166
0.25870752334594727
0

0.16179853677749634
0.2152451127767563
0.0594804584980011
0.5767977833747864
0.3624415397644043
0.31916332244873047
0.24383488297462463
0.23012268543243408
0.2229629009962082
0.18652093410491943
0.17278409004211426
0.16141612827777863
0.13136334717273712
0.11762882769107819
0.09530399739742279
0.05892712250351906
0.058595433831214905
0.05801079422235489
0.05676155164837837
0.1324368566274643
0.14606156945228577
0.10417134314775467
0.05497074872255325
0.059730712324380875
0.0532299242913723
0.05237269401550293
0.10787584632635117
0.08272314071655273
0.08024658262729645
0.07045216113328934
0.058358266949653625
0.056961871683597565
0.053671810775995255
0.05017974600195885
0.6760759353637695
0.07037530094385147
0.06431066989898682
0.06385137140750885
0.07109420746564865
0.06575462967157364
0.05298677086830139
0.06181170791387558
0.11699789017438889
0.9306817650794983
0.4234686493873596
0.39744657278060913
0.390432208776474
0.07075069099664688
0.0624275729060173
0.05691460520029068
0.222448

...

0.061752595007419586
0.06083814799785614
0.05660708621144295
0.05645092576742172
0.05604289472103119
0.055733468383550644
0.0546007864177227
0.053637873381376266
0.05342783033847809
0.05287961661815643
0.40882739424705505
0.05029003694653511
0.500841498374939
0.0666840597987175
0.16112931072711945
0.9914912581443787
0.8097860813140869
0.7696100473403931
0.502784252166748
0.3211473524570465
0.26399239897727966
0.17770852148532867
0.14969493448734283
0.14166730642318726
0.10676801949739456
0.081626296043396
0.07649371773004532
0.06278956681489944
0.06139347702264786
0.05317021161317825
0.05873991176486015
0.5905845761299133
0.14585821330547333
0.10994214564561844
0.08171999454498291
0.07759549468755722
0.07654576748609543
0.05370586737990379
0.9975563287734985
0.8671277165412903
0.05998750403523445
0.09537892043590546
0.05274648219347
0.05366866663098335
0.9949072003364563
0.8034213781356812
0.214934840798378
0.09318702667951584
0.06660496443510056
0.9131540060043335
0.9893538951873779
0

0.9566254019737244
0.49088650941848755
0.4150846600532532
0.2569597363471985
0.16228683292865753
0.14111536741256714
0.1390075534582138
0.12444930523633957
0.1235533058643341
0.11448114365339279
0.10165923833847046
0.09011975675821304
0.08636016398668289
0.07889282703399658
0.07869194447994232
0.07665196806192398
0.07489682734012604
0.06575457006692886
0.06533505022525787
0.0641331747174263
0.06223415955901146
0.05710649490356445
0.05323883146047592
0.051656901836395264
0.12736214697360992
0.09984426200389862
0.07380516827106476
0.24906186759471893
0.05411858856678009
0.05007348582148552
0.055700112134218216
0.4074109196662903
0.1949606090784073
0.06690816581249237
0.0633629709482193
0.0516725555062294
0.31420618295669556
0.08307833969593048
0.06574021279811859
0.13291166722774506
0.09031329303979874
0.05463479459285736
0.1119404137134552
0.06590495258569717
0.05679595097899437
0.19544653594493866
0.08037638664245605
0.06321833282709122
0.05039495602250099
0.9742766618728638
0.13105253

0.11472716927528381
0.11369939893484116
0.09640449285507202
0.0875762477517128
0.07215413451194763
0.061763785779476166
0.05594464763998985
0.05444715544581413
0.053944140672683716
0.1475309282541275
0.7733328342437744
0.4866097569465637
0.26923948526382446
0.14699441194534302
0.13345155119895935
0.1019371822476387
0.09685903787612915
0.08881532400846481
0.08437536656856537
0.07074949890375137
0.0683484822511673
0.06620818376541138
0.060857851058244705
0.05790520831942558
0.056243009865283966
0.05068374052643776
0.06656432151794434
0.052808672189712524
0.05236899107694626
0.2824755907058716
0.13766354322433472
0.9762675166130066
0.07115780562162399
0.07082483917474747
0.05446033552289009
0.1163964718580246
0.21954229474067688
0.07034250348806381
0.06821902096271515
0.053479474037885666
0.06590362638235092
0.05791311338543892
0.05651828274130821
0.0907997190952301
0.06683635711669922
0.06334269046783447
0.24431847035884857
0.21454748511314392
0.18547065556049347
0.12473130971193314
0.09

0.052607305347919464
0.06341198831796646
0.11832508444786072
0.9959897398948669
0.9491315484046936
0.060263875871896744
0.8978097438812256
0.9620772004127502
0.0735272616147995
0.06528522819280624
0.6617226004600525
0.8602457642555237
0.7087255716323853
0.07052270323038101
0.29962942004203796
0.880922257900238
0.11162737756967545
0.4024707078933716
0.07853355258703232
0.9917196035385132
0.055475879460573196
0.2187492549419403
0.09164226055145264
0.08006726950407028
0.9400908350944519
0.05024268105626106
0.12235007435083389
0.06470028311014175
0.9126308560371399
0.1495353877544403
0.5041359663009644
0.1844286173582077
0.08018365502357483
0.15663771331310272
0.1418217122554779
0.9381269216537476
0.7557564377784729
0.20002694427967072
0.13964207470417023
0.05808883532881737
0.5592076778411865
0.9739202857017517
0.20773202180862427
0.3549756407737732
0.17337648570537567
0.6676226258277893
0.14237399399280548
0.11981133371591568
0.06420040130615234
0.15640774369239807
0.06148064136505127
0.

0.06272462010383606
0.9988767504692078
0.32108819484710693
0.9980294108390808
0.25369301438331604
0.12834008038043976
0.3747159540653229
0.17628973722457886
0.10803695023059845
0.06971290707588196
0.9903383851051331
0.44527095556259155
0.09722086042165756
0.05829586461186409
0.11123521625995636
0.9948194622993469
0.17434430122375488
0.06707825511693954
0.15952445566654205
0.05210283771157265
0.9369038343429565
0.9980060458183289
0.9750341773033142
0.05326275900006294
0.9911659359931946
0.15692071616649628
0.06523440033197403
0.0640677958726883
0.06170634180307388
0.051943618804216385
0.08067251741886139
0.06671904027462006
0.12176859378814697
0.9648787975311279
0.9860726594924927
0.06600438058376312
0.9997885823249817
0.1118897944688797
0.05706927180290222
0.9994945526123047
0.75782710313797
0.06634491682052612
0.9992714524269104
0.6089551448822021
0.17337174713611603
0.9994839429855347
0.9890690445899963
0.9591313600540161
0.10019896924495697
0.9982181191444397
0.1657528132200241
0.06

# evaluation

In [92]:
def python_eval(output_dir='output'):
        annopath = os.path.join(
            DEVKIT_PATH,
            'VOC2007',
            'Annotations',
            '{:s}.xml')
        imagesetfile = os.path.join(
            DEVKIT_PATH,
            'VOC2007',
            'ImageSets',
            'Main',
            SET + '.txt')
        cachedir = os.path.join(DEVKIT_PATH, 'annotations_cache')
        aps = []
        # The PASCAL VOC metric changed in 2010.
        # VOC07 metric is quite old so don't use.
        use_07_metric = False
        print('VOC07 metric? ' + ('Yes' if use_07_metric else 'No'))
        if not os.path.isdir(output_dir):
            os.mkdir(output_dir)
        for i, cls in enumerate(PASCAL_CLASSES):
            if cls == 'bg':
                continue
            filename = get_voc_results_file_template(cls)
            rec, prec, ap = voc_eval(
                filename, annopath, imagesetfile, cls, cachedir, ovthresh=0.5,
                use_07_metric=use_07_metric)
            aps += [ap]
            print('AP for {} = {:.4f}'.format(cls, ap))
            with open(os.path.join(output_dir, cls + '_pr.pkl'), 'wb') as f:
                pickle.dump({'rec': rec, 'prec': prec, 'ap': ap}, f)
        print('Mean AP = {:.4f}'.format(np.mean(aps)))
        print('~~~~~~~~')
        print('Results:')
        for ap in aps:
            print('{:.3f}'.format(ap))
        print('{:.3f}'.format(np.mean(aps)))
        print('~~~~~~~~')
        print('')
        print('--------------------------------------------------------------')
        print('Results computed with the **unofficial** Python eval code.')
        print('Results should be very close to the official MATLAB eval code.')
        print('Recompute with `./tools/reval.py --matlab ...` for your paper.')
        print('-- Thanks, The Management')
        print('--------------------------------------------------------------')

In [93]:
# evaluate detections
python_eval()

VOC07 metric? No
Reading annotation for 1/4952
Reading annotation for 101/4952
Reading annotation for 201/4952
Reading annotation for 301/4952
Reading annotation for 401/4952
Reading annotation for 501/4952
Reading annotation for 601/4952
Reading annotation for 701/4952
Reading annotation for 801/4952
Reading annotation for 901/4952
Reading annotation for 1001/4952
Reading annotation for 1101/4952
Reading annotation for 1201/4952
Reading annotation for 1301/4952
Reading annotation for 1401/4952
Reading annotation for 1501/4952
Reading annotation for 1601/4952
Reading annotation for 1701/4952
Reading annotation for 1801/4952
Reading annotation for 1901/4952
Reading annotation for 2001/4952
Reading annotation for 2101/4952
Reading annotation for 2201/4952
Reading annotation for 2301/4952
Reading annotation for 2401/4952
Reading annotation for 2501/4952
Reading annotation for 2601/4952
Reading annotation for 2701/4952
Reading annotation for 2801/4952
Reading annotation for 2901/4952
Readi

AP for bottle = 0.5177
Reading annotation for 1/4952
Reading annotation for 101/4952
Reading annotation for 201/4952
Reading annotation for 301/4952
Reading annotation for 401/4952
Reading annotation for 501/4952
Reading annotation for 601/4952
Reading annotation for 701/4952
Reading annotation for 801/4952
Reading annotation for 901/4952
Reading annotation for 1001/4952
Reading annotation for 1101/4952
Reading annotation for 1201/4952
Reading annotation for 1301/4952
Reading annotation for 1401/4952
Reading annotation for 1501/4952
Reading annotation for 1601/4952
Reading annotation for 1701/4952
Reading annotation for 1801/4952
Reading annotation for 1901/4952
Reading annotation for 2001/4952
Reading annotation for 2101/4952
Reading annotation for 2201/4952
Reading annotation for 2301/4952
Reading annotation for 2401/4952
Reading annotation for 2501/4952
Reading annotation for 2601/4952
Reading annotation for 2701/4952
Reading annotation for 2801/4952
Reading annotation for 2901/4952

Reading annotation for 401/4952
Reading annotation for 501/4952
Reading annotation for 601/4952
Reading annotation for 701/4952
Reading annotation for 801/4952
Reading annotation for 901/4952
Reading annotation for 1001/4952
Reading annotation for 1101/4952
Reading annotation for 1201/4952
Reading annotation for 1301/4952
Reading annotation for 1401/4952
Reading annotation for 1501/4952
Reading annotation for 1601/4952
Reading annotation for 1701/4952
Reading annotation for 1801/4952
Reading annotation for 1901/4952
Reading annotation for 2001/4952
Reading annotation for 2101/4952
Reading annotation for 2201/4952
Reading annotation for 2301/4952
Reading annotation for 2401/4952
Reading annotation for 2501/4952
Reading annotation for 2601/4952
Reading annotation for 2701/4952
Reading annotation for 2801/4952
Reading annotation for 2901/4952
Reading annotation for 3001/4952
Reading annotation for 3101/4952
Reading annotation for 3201/4952
Reading annotation for 3301/4952
Reading annotati

Reading annotation for 2301/4952
Reading annotation for 2401/4952
Reading annotation for 2501/4952
Reading annotation for 2601/4952
Reading annotation for 2701/4952
Reading annotation for 2801/4952
Reading annotation for 2901/4952
Reading annotation for 3001/4952
Reading annotation for 3101/4952
Reading annotation for 3201/4952
Reading annotation for 3301/4952
Reading annotation for 3401/4952
Reading annotation for 3501/4952
Reading annotation for 3601/4952
Reading annotation for 3701/4952
Reading annotation for 3801/4952
Reading annotation for 3901/4952
Reading annotation for 4001/4952
Reading annotation for 4101/4952
Reading annotation for 4201/4952
Reading annotation for 4301/4952
Reading annotation for 4401/4952
Reading annotation for 4501/4952
Reading annotation for 4601/4952
Reading annotation for 4701/4952
Reading annotation for 4801/4952
Reading annotation for 4901/4952
AP for pottedplant = 0.5207
Reading annotation for 1/4952
Reading annotation for 101/4952
Reading annotation 